<div style="text-align: center; font-size: 25px;">
Trực quan hóa dữ liệu - CQ2021/21 <br>
Nhóm 02<br>
TLab 02: Working with Time-series data <br>
Chủ đề: Visualize about the climate in Indonesia
</div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

df_climate = pd.read_csv('climate_data.csv')
df_province = pd.read_csv('province_detail.csv')
df_station = pd.read_csv('station_detail.csv')

# merge 3 csv
df_merg = df_station.merge(df_province, on=["province_id"], how='left')
df_merg_last = df_climate.merge(df_merg, on=["station_id"], how='left')
df = df_merg_last.copy()
df.head()

,date,Tn,Tx,Tavg,RH_avg,RR,ss,ff_x,ddd_x,ff_avg,ddd_car,station_id,station_name,region_name,latitude,longitude,region_id,province_id,province_name
0,01-01-2010,21.4,30.2,27.1,82.0,9.0,0.5,7.0,90.0,5.0,E,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam
1,02-01-2010,21.0,29.6,25.7,95.0,24.0,0.2,6.0,90.0,4.0,E,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam
2,03-01-2010,20.2,26.8,24.5,98.0,63.0,0.0,5.0,90.0,4.0,E,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam
3,04-01-2010,21.0,29.2,25.8,90.0,0.0,0.1,4.0,225.0,3.0,SW,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam
4,05-01-2010,21.2,30.0,26.7,90.0,2.0,0.4,NaN,NaN,NaN,NaN,96001,Stasiun Meteorologi Maimun Saleh,Kota Sabang,5.87655,95.33785,20,1,Nanggroe Aceh Darussalam


# A. Thu thập dữ liệu

* Ngữ cảnh:
    ...

* Bộ dữ liệu:
    ...

* License:
    ...

* Cách thức thu thập dữ liệu:
    ...

# B. Khám phá dữ liệu

# C. Khám phá mối quan hệ trong dữ liệu

### Temperature

### Humid

### Rain

### Sunshine

### Windspeed